**<h2> Importing Libraries**

In [28]:
import json
import os
import tensorflow as tf


**<h2>Uploading Caption File**

In [2]:
path=r'/home/tarushi/Desktop/Img-Caption/dataset/annotations_trainval2014/annotations/captions_train2014.json'

In [3]:
f=open(path)
data=json.load(f)

**<h2>Extracting Captions**
We'll now extract the captions of the images. Each Image has been provided with five captions.

I'll be extracting the captions for the images I have. 
So that I don't encounter an 'image not found' error,I'll list out the images stored in my system and match their names with the image ids of the annotation provided to get captions for those particular images.

In [18]:
path_imgs=r'/home/tarushi/Desktop/Img-Caption/images/train2014'
img_list=os.listdir(path_imgs)
img_list=img_list[:3000]
img_list[:10]

['COCO_train2014_000000105728.jpg',
 'COCO_train2014_000000175053.jpg',
 'COCO_train2014_000000047249.jpg',
 'COCO_train2014_000000264241.jpg',
 'COCO_train2014_000000162693.jpg',
 'COCO_train2014_000000204785.jpg',
 'COCO_train2014_000000203976.jpg',
 'COCO_train2014_000000223461.jpg',
 'COCO_train2014_000000057300.jpg',
 'COCO_train2014_000000147250.jpg']

In [54]:
all_captions=[]
for i in data['annotations']:
    img_id='COCO_train2014_' + '%012d.jpg' % (i['image_id'])
    if img_id in img_list:
        all_captions.append(['<start>'+i['caption']+'<end>',img_id])


Next,I'll sort out the obtained captions according to the images.

In [55]:
all_captions= sorted(all_captions,key=lambda x:x[1])
all_captions[:2]

[['<start>a man and woman cut into a big cake <end>',
  'COCO_train2014_000000000113.jpg'],
 ['<start>A man and woman standing in front of a cake.<end>',
  'COCO_train2014_000000000113.jpg']]

In [56]:
sorted_captions=[all_captions[i][0] for i in range(len(all_captions))]
sorted_captions[:2]

['<start>a man and woman cut into a big cake <end>',
 '<start>A man and woman standing in front of a cake.<end>']

In [57]:
len(sorted_captions)

15002

**<h2>Preprocessing and Tokenizing the Images**

In [58]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


**<h3>Tokenizing**

In [59]:
# Choose the top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(all_captions)
train_seqs = tokenizer.texts_to_sequences(all_captions)


In [32]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


**<h3>Padding the Sequences**


In [33]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(all_captions)
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
max_length = calc_max_length(train_seqs)


In [34]:
print(max_length)

51
